> Note: If you have a different iSamples PQG parquet file from another provider, set `file_url` and `LOCAL_PATH` accordingly. All queries below will still work because they rely on PQG structure and iSamples model semantics.

# iSamples PQG Parquet Analysis (using OpenContext dataset)

This notebook analyzes an iSamples Property Graph (PQG) parquet file. The sample file we use happens to be produced from OpenContext, but the schema, node types, and graph patterns are iSamples‑generic.

## Key Distinction: PQG framework vs iSamples model vs provider data

We’ll keep these layers straight:

1. Generic PQG (Property Graph) framework
   - Core graph fields: `s` (subject), `p` (predicate), `o` (object array), `n` (graph name)
   - Edges are rows with `otype = '_edge_'`
   - Graph traversal patterns (joins on s/p/o) are domain‑agnostic

2. iSamples metadata model (provider‑agnostic domain schema)
   - Entity types: `MaterialSampleRecord`, `SamplingEvent`, `GeospatialCoordLocation`, `SamplingSite`, `IdentifiedConcept`, `Agent`, etc.
   - Predicates like `produced_by`, `sample_location`, `sampling_site`, `has_material_category`, etc.
   - These are defined by the iSamples model, not specific to OpenContext

3. Provider data (e.g., OpenContext)
   - A particular provider’s content fills the iSamples model
   - The dataset URL we load is from OpenContext, but the analysis is reusable for any iSamples PQG parquet

## Setup and Data Loading

In [1]:
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import urllib.request
import os

# Configuration
file_url = "https://storage.googleapis.com/opencontext-parquet/oc_isamples_pqg.parquet"
# LOCAL_PATH is configured for Raymond Yee's local machine 

# ~/Data/iSample/pqg_refining/oc_isamples_pqg.parquet
LOCAL_PATH = os.path.join(Path.home(), "Data", "iSample", "oc_isamples_pqg.parquet")

In [2]:
# Check if local file exists, download to generic location if not

if not os.path.exists(LOCAL_PATH):
    print(f"Local file not found at {LOCAL_PATH}")

    # if the file is not there, let's use tempfile module to create a temp file path.
    # put tempfile in /tmp/oc_isamples_pqg.parquet
    LOCAL_PATH = os.path.join("/tmp", "oc_isamples_pqg.parquet")
    os.makedirs(os.path.dirname(LOCAL_PATH), exist_ok=True)
    
    print(f"Downloading {file_url} to {LOCAL_PATH}...")
    urllib.request.urlretrieve(file_url, LOCAL_PATH)
    print("Download completed!")
else:
    print(f"Local file already exists at {LOCAL_PATH}")

# Use local path for parquet operations
parquet_path = LOCAL_PATH
print(f"Using parquet file: {parquet_path}")

Local file not found at /Users/raymondyee/Data/iSample/pqg_refining/oc_isamples_pqg.parquet
Download completed!
Using parquet file: /tmp/oc_isamples_pqg.parquet


In [3]:
# Create a DuckDB connection
conn = duckdb.connect()

# Create view for the parquet file
conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")

# Count records
result = conn.execute("SELECT COUNT(*) FROM pqg;").fetchone()
print(f"Total records: {result[0]:,}")

Total records: 11,786,509


In [4]:
# Schema information
print("Schema information:")
schema_result = conn.execute("DESCRIBE pqg;").fetchall()
for row in schema_result[:10]:  # Show first 10 columns
    print(f"{row[0]:25} | {row[1]}")
print(f"... and {len(schema_result) - 10} more columns")

Schema information:
row_id                    | INTEGER
pid                       | VARCHAR
tcreated                  | INTEGER
tmodified                 | INTEGER
otype                     | VARCHAR
s                         | INTEGER
p                         | VARCHAR
o                         | INTEGER[]
n                         | VARCHAR
altids                    | VARCHAR[]
... and 30 more columns


In [5]:
# Examine the distribution of entity types (iSamples model types)
entity_stats = conn.execute("""
    SELECT
        otype,
        COUNT(*) as count,
        COUNT(DISTINCT pid) as unique_pids,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
    FROM pqg
    GROUP BY otype
    ORDER BY count DESC
""").fetchdf()

print("Entity Type Distribution (iSamples model types):")
print(entity_stats)

Entity Type Distribution (iSamples model types):
                     otype    count  unique_pids  percentage
0                   _edge_  9321819      9321819       79.09
1     MaterialSampleRecord  1110412      1110412        9.42
2            SamplingEvent  1110412      1110412        9.42
3  GeospatialCoordLocation   199147       199147        1.69
4        IdentifiedConcept    25929        25929        0.22
5             SamplingSite    18213        18213        0.15
6                    Agent      577          577        0.00


## Complete Analysis: All Sentence Types in the Dataset

Now let's compute ALL sentence types (subject-predicate-object patterns) in the parquet file, following the pattern we discovered:

In [6]:
# COMPUTE ALL SENTENCE TYPES: Subject -> Predicate -> Object patterns
print("ALL SENTENCE TYPES IN THE DATASET")
print("=" * 50)

# For each predicate, find what subject types use it and what object types they point to
all_sentence_types = conn.execute("""
    WITH relationship_data AS (
        -- Get all relationships (where p is not null)
        SELECT s, p, o[1] as target_id
        FROM pqg 
        WHERE p IS NOT NULL AND array_length(o) > 0
    ),
    subject_entities AS (
        -- Get subject entity types
        SELECT row_id, otype as subject_type
        FROM pqg 
        WHERE p IS NULL
    ),
    object_entities AS (
        -- Get object entity types  
        SELECT row_id, otype as object_type
        FROM pqg 
        WHERE p IS NULL
    )
    SELECT 
        subject_entities.subject_type,
        relationship_data.p as predicate,
        object_entities.object_type,
        COUNT(*) as frequency,
        COUNT(DISTINCT relationship_data.s) as unique_subjects,
        COUNT(DISTINCT relationship_data.target_id) as unique_objects
    FROM relationship_data
    JOIN subject_entities ON relationship_data.s = subject_entities.row_id
    JOIN object_entities ON relationship_data.target_id = object_entities.row_id
    GROUP BY subject_entities.subject_type, relationship_data.p, object_entities.object_type
    ORDER BY frequency DESC
""").fetchdf()

print(f"Found {len(all_sentence_types)} unique sentence types:")
print()

# Display all sentence types
for idx, row in all_sentence_types.iterrows():
    subject = row['subject_type']
    predicate = row['predicate']
    object_type = row['object_type']
    freq = row['frequency']
    unique_subj = row['unique_subjects']
    unique_obj = row['unique_objects']
    
    print(f"{subject} --{predicate}--> {object_type}")
    print(f"  Frequency: {freq:,} relationships")
    print(f"  Unique subjects: {unique_subj:,}, Unique objects: {unique_obj:,}")
    print()

ALL SENTENCE TYPES IN THE DATASET
Found 10 unique sentence types:

MaterialSampleRecord --produced_by--> SamplingEvent
  Frequency: 1,110,412 relationships
  Unique subjects: 1,110,412, Unique objects: 1,110,412

MaterialSampleRecord --has_sample_object_type--> IdentifiedConcept
  Frequency: 1,110,412 relationships
  Unique subjects: 1,110,412, Unique objects: 5

MaterialSampleRecord --has_material_category--> IdentifiedConcept
  Frequency: 1,110,412 relationships
  Unique subjects: 1,110,412, Unique objects: 10

MaterialSampleRecord --has_context_category--> IdentifiedConcept
  Frequency: 1,110,412 relationships
  Unique subjects: 1,110,412, Unique objects: 2

SamplingEvent --sampling_site--> SamplingSite
  Frequency: 1,110,412 relationships
  Unique subjects: 1,110,412, Unique objects: 18,213

MaterialSampleRecord --keywords--> IdentifiedConcept
  Frequency: 1,110,359 relationships
  Unique subjects: 1,110,359, Unique objects: 2,311

SamplingEvent --sample_location--> GeospatialCoord

In [7]:
# SENTENCE TYPE ANALYSIS SUMMARY
print("SENTENCE TYPE ANALYSIS")
print("=" * 40)

# Group sentence types by subject type
subject_groups = all_sentence_types.groupby('subject_type').agg({
    'predicate': 'count',
    'frequency': 'sum'
}).rename(columns={'predicate': 'num_predicates', 'frequency': 'total_relationships'})

print("Sentence types grouped by SUBJECT:")
for subject, data in subject_groups.iterrows():
    predicates_for_subject = all_sentence_types[all_sentence_types['subject_type'] == subject]['predicate'].tolist()
    print(f"\n{subject}:")
    print(f"  Uses {data['num_predicates']} different predicates: {', '.join(predicates_for_subject)}")
    print(f"  Total relationships: {data['total_relationships']:,}")

print(f"\n\nSENTENCE TYPE PATTERNS:")
print("-" * 30)

# Analyze the patterns
print("1. MaterialSampleRecord entities have the most relationships:")
msr_types = all_sentence_types[all_sentence_types['subject_type'] == 'MaterialSampleRecord']
for _, row in msr_types.iterrows():
    print(f"   - {row['predicate']} → {row['object_type']} ({row['frequency']:,} times)")

print(f"\n2. SamplingEvent entities connect to:")
se_types = all_sentence_types[all_sentence_types['subject_type'] == 'SamplingEvent']
for _, row in se_types.iterrows():
    print(f"   - {row['predicate']} → {row['object_type']} ({row['frequency']:,} times)")

print(f"\n3. SamplingSite entities:")
ss_types = all_sentence_types[all_sentence_types['subject_type'] == 'SamplingSite']
for _, row in ss_types.iterrows():
    print(f"   - {row['predicate']} → {row['object_type']} ({row['frequency']:,} times)")

print(f"\nTOTAL: {len(all_sentence_types)} unique sentence types define the complete grammar of this dataset!")

SENTENCE TYPE ANALYSIS
Sentence types grouped by SUBJECT:

MaterialSampleRecord:
  Uses 6 different predicates: produced_by, has_sample_object_type, has_material_category, has_context_category, keywords, registrant
  Total relationships: 5,973,528

SamplingEvent:
  Uses 3 different predicates: sampling_site, sample_location, responsibility
  Total relationships: 3,330,078

SamplingSite:
  Uses 1 different predicates: site_location
  Total relationships: 18,213


SENTENCE TYPE PATTERNS:
------------------------------
1. MaterialSampleRecord entities have the most relationships:
   - produced_by → SamplingEvent (1,110,412 times)
   - has_sample_object_type → IdentifiedConcept (1,110,412 times)
   - has_material_category → IdentifiedConcept (1,110,412 times)
   - has_context_category → IdentifiedConcept (1,110,412 times)
   - keywords → IdentifiedConcept (1,110,359 times)
   - registrant → Agent (421,521 times)

2. SamplingEvent entities connect to:
   - sampling_site → SamplingSite (1,11

### Theoretical Edge Sentence Types from LinkML Schema

To understand how many edge sentence types are theoretically possible, we parse the actual iSamples LinkML schema and extract only the relationships that create edges between entities (not properties like strings or dates):

In [8]:
# Parse the iSamples LinkML schema to derive theoretical edge sentence types
import yaml
import urllib.request

# Use the live GitHub schema instead of local file
schema_url = "https://raw.githubusercontent.com/isamplesorg/metadata/refs/heads/main/src/schemas/isamples_core.yaml"

print(f"✅ Reading live iSamples LinkML schema from GitHub: {schema_url}")

try:
    # Download and parse the LinkML schema from GitHub
    with urllib.request.urlopen(schema_url) as response:
        schema_content = response.read().decode('utf-8')
    
    schema = yaml.safe_load(schema_content)
    
    # Extract all relationships from classes and their slots
    all_theoretical_relationships = []
    classes = schema.get('classes', {})
    
    for class_name, class_def in classes.items():
        slots = class_def.get('slots', [])
        for slot_name in slots:
            # Look up slot definition to get target type
            slot_def = schema.get('slots', {}).get(slot_name, {})
            slot_range = slot_def.get('range', 'Unknown')
            all_theoretical_relationships.append((class_name, slot_name, slot_range))
    
    print(f"📋 Extracted {len(all_theoretical_relationships)} total relationships from live schema")
    
    # Filter to edge relationships only (entity → entity, not entity → primitive)
    # Updated with all entity classes found in the live schema
    isamples_entity_classes = {
        'MaterialSampleRecord', 'SamplingEvent', 'SamplingSite', 'GeospatialCoordLocation',
        'Agent', 'IdentifiedConcept', 'MaterialSampleCuration', 'SampleRelation'
    }
    
    theoretical_edge_relationships = [
        (subj, pred, obj) for subj, pred, obj in all_theoretical_relationships 
        if obj in isamples_entity_classes
    ]
    
    print(f"🔗 Filtered to {len(theoretical_edge_relationships)} edge relationships (entity→entity)")
    THEORETICAL_RELATIONSHIPS = [(s, p, o, f"{s} → {o}") for s, p, o in theoretical_edge_relationships]
    
    print(f"\n🎯 Found {len(theoretical_edge_relationships)} theoretical edge sentence types from live schema")
    
except Exception as e:
    print(f"❌ Error reading schema from GitHub: {e}")
    print("Using fallback - cannot derive from actual schema")
    theoretical_edge_relationships = []
    THEORETICAL_RELATIONSHIPS = []

✅ Reading live iSamples LinkML schema from GitHub: https://raw.githubusercontent.com/isamplesorg/metadata/refs/heads/main/src/schemas/isamples_core.yaml
📋 Extracted 57 total relationships from live schema
🔗 Filtered to 14 edge relationships (entity→entity)

🎯 Found 14 theoretical edge sentence types from live schema


In [9]:
# Let's systematically compute ALL possible (subject_type, predicate, object_type) sentence types
# where otype = '_edge_' (actual graph edges, not literals)

print("🔬 **SYSTEMATIC ANALYSIS: All Possible EDGE Sentence Types**")
print("=" * 70)

# First, let's see what we actually extracted from the LinkML schema
print(f"✅ Loaded {len(THEORETICAL_RELATIONSHIPS)} total theoretical relationships from LinkML schema")

# The key insight: we only want EDGE relationships (entity → entity), not entity → literal
# In parquet files, edges have otype='_edge_' and connect entities, while literals are stored as entity properties

# Use the same entity classes as defined in the schema parsing cell for consistency
print(f"\n📋 **iSamples Entity Classes**: {sorted(isamples_entity_classes)}")

# Filter to only edge relationships (entity → entity, not entity → literal/primitive)
theoretical_edge_relationships = []
for subj, pred, obj, desc in THEORETICAL_RELATIONSHIPS:
    if obj in isamples_entity_classes:  # Object must be an entity class
        theoretical_edge_relationships.append((subj, pred, obj))

print(f"\n🔗 **Theoretical EDGE Relationships** (entity→entity): {len(theoretical_edge_relationships)}")
for i, (subj, pred, obj) in enumerate(theoretical_edge_relationships, 1):
    print(f"{i:2d}. ({subj}, {pred}, {obj})")

# What we discovered in OpenContext data (all are edges since we filtered to otype='_edge_')
discovered_edge_types = set()
for _, row in all_sentence_types.iterrows():
    subj_type = row['subject_type']
    predicate = row['predicate'] 
    obj_type = row['object_type']
    discovered_edge_types.add((subj_type, predicate, obj_type))

print(f"\n✅ **Discovered in OpenContext** (all are edges): {len(discovered_edge_types)}")
for i, (subj, pred, obj) in enumerate(sorted(discovered_edge_types), 1):
    # Get frequency
    matching = all_sentence_types[
        (all_sentence_types['subject_type'] == subj) & 
        (all_sentence_types['predicate'] == pred) & 
        (all_sentence_types['object_type'] == obj)
    ]
    freq = matching['frequency'].iloc[0] if not matching.empty else 0
    print(f"{i:2d}. ({subj}, {pred}, {obj}) → {freq:,} instances")

# Find missing theoretical edge relationships
theoretical_edge_tuples = set(theoretical_edge_relationships)
missing_edge_relationships = theoretical_edge_tuples - discovered_edge_types
implemented_edge_relationships = theoretical_edge_tuples & discovered_edge_types

print(f"\n❌ **MISSING** theoretical edge sentence types ({len(missing_edge_relationships)}):")
for i, (subj, pred, obj) in enumerate(sorted(missing_edge_relationships), 1):
    print(f"{i:2d}. ({subj}, {pred}, {obj})")

print(f"\n✅ **IMPLEMENTED** theoretical edge sentence types ({len(implemented_edge_relationships)}):")
for i, (subj, pred, obj) in enumerate(sorted(implemented_edge_relationships), 1):
    print(f"{i:2d}. ({subj}, {pred}, {obj})")

# Check for bonus edge relationships (found but not in theory)
bonus_edge_relationships = discovered_edge_types - theoretical_edge_tuples
if bonus_edge_relationships:
    print(f"\n🎁 **BONUS** edge relationships ({len(bonus_edge_relationships)} not in LinkML schema):")
    for i, (subj, pred, obj) in enumerate(sorted(bonus_edge_relationships), 1):
        print(f"{i:2d}. ({subj}, {pred}, {obj})")

print(f"\n📊 **SUMMARY: Edge Sentence Types Only**")
print(f"• Theoretical edge sentence types: {len(theoretical_edge_relationships)}")
print(f"• Discovered in OpenContext: {len(discovered_edge_types)}")
print(f"• Implementation coverage: {len(implemented_edge_relationships)}/{len(theoretical_edge_relationships)} = {100*len(implemented_edge_relationships)/len(theoretical_edge_relationships):.1f}%")
print(f"• Missing from OpenContext: {len(missing_edge_relationships)}")
print(f"• Bonus patterns in OpenContext: {len(bonus_edge_relationships)}")

# Store results for further analysis
THEORETICAL_EDGE_COUNT = len(theoretical_edge_relationships)
DISCOVERED_EDGE_COUNT = len(discovered_edge_types)
COVERAGE_PERCENTAGE = 100 * len(implemented_edge_relationships) / len(theoretical_edge_relationships)

print(f"\n🎯 **Answer**: There are {THEORETICAL_EDGE_COUNT} theoretically possible EDGE sentence types")
print(f"   OpenContext implements {DISCOVERED_EDGE_COUNT} of them ({COVERAGE_PERCENTAGE:.1f}% coverage)")
print(f"   All OpenContext sentence types are valid edges (otype='_edge_') connecting entities")

🔬 **SYSTEMATIC ANALYSIS: All Possible EDGE Sentence Types**
✅ Loaded 14 total theoretical relationships from LinkML schema

📋 **iSamples Entity Classes**: ['Agent', 'GeospatialCoordLocation', 'IdentifiedConcept', 'MaterialSampleCuration', 'MaterialSampleRecord', 'SampleRelation', 'SamplingEvent', 'SamplingSite']

🔗 **Theoretical EDGE Relationships** (entity→entity): 14
 1. (MaterialSampleRecord, produced_by, SamplingEvent)
 2. (MaterialSampleRecord, has_context_category, IdentifiedConcept)
 3. (MaterialSampleRecord, has_material_category, IdentifiedConcept)
 4. (MaterialSampleRecord, has_sample_object_type, IdentifiedConcept)
 5. (MaterialSampleRecord, keywords, IdentifiedConcept)
 6. (MaterialSampleRecord, related_resource, SampleRelation)
 7. (MaterialSampleRecord, curation, MaterialSampleCuration)
 8. (MaterialSampleRecord, registrant, Agent)
 9. (SamplingSite, site_location, GeospatialCoordLocation)
10. (SamplingEvent, has_context_category, IdentifiedConcept)
11. (SamplingEvent, re

## PQG Grammar Analysis: 14 Theoretical Edge Sentence Types

Looking at the **fundamental PQG grammar** derived from the live iSamples LinkML schema:

### Complete Theoretical Grammar (14 Edge Types)
The iSamples model defines **14 theoretically possible edge sentence types** (entity→entity relationships):

**MaterialSampleRecord relationships (7):**
1. `MaterialSampleRecord → SamplingEvent` (produced_by) 
2. `MaterialSampleRecord → IdentifiedConcept` (has_context_category)
3. `MaterialSampleRecord → IdentifiedConcept` (has_material_category) 
4. `MaterialSampleRecord → IdentifiedConcept` (has_sample_object_type)
5. `MaterialSampleRecord → IdentifiedConcept` (keywords)
6. `MaterialSampleRecord → SampleRelation` (related_resource) 
7. `MaterialSampleRecord → MaterialSampleCuration` (curation)
8. `MaterialSampleRecord → Agent` (registrant)

**SamplingEvent relationships (4):**
9. `SamplingEvent → IdentifiedConcept` (has_context_category)
10. `SamplingEvent → Agent` (responsibility)
11. `SamplingEvent → SamplingSite` (sampling_site)
12. `SamplingEvent → GeospatialCoordLocation` (sample_location)

**SamplingSite relationships (1):**
13. `SamplingSite → GeospatialCoordLocation` (site_location)

**MaterialSampleCuration relationships (1):**
14. `MaterialSampleCuration → Agent` (responsibility)

### OpenContext Implementation: 10/14 (71.4% coverage)

OpenContext implements 10 of these 14 theoretical edge types, missing:
- Curation workflows (`MaterialSampleCuration` relationships)
- Cross-sample references (`SampleRelation` relationships)  
- Event context classification (`SamplingEvent → has_context_category`)
- Resource linking (`MaterialSampleRecord → related_resource`)

This represents a **complete PQG grammar analysis** showing that **14 edge sentence types** are theoretically possible in the iSamples model.

In [10]:
# Explore edge predicates (iSamples model predicates)
edge_predicates = conn.execute("""
    SELECT
        p as predicate,
        COUNT(*) as usage_count,
        COUNT(DISTINCT s) as unique_subjects
    FROM pqg
    WHERE otype = '_edge_'
    GROUP BY p
    ORDER BY usage_count DESC
""").fetchdf()

print("Most common relationship types (iSamples predicates):")
print(edge_predicates)

Most common relationship types (iSamples predicates):
                predicate  usage_count  unique_subjects
0   has_material_category      1110412          1110412
1    has_context_category      1110412          1110412
2           sampling_site      1110412          1110412
3  has_sample_object_type      1110412          1110412
4             produced_by      1110412          1110412
5                keywords      1110359          1110359
6         sample_location      1110334          1110334
7          responsibility      1109332          1109332
8              registrant       421521           421521
9           site_location        18213            18213


In [11]:
# First, let's check what columns we actually have in the dataset
schema_check = conn.execute("""
    SELECT * FROM pqg LIMIT 1
""").fetchdf()

print("Available columns:")
print(schema_check.columns.tolist())
print("\nSample row:")
print(schema_check.iloc[0])

Available columns:
['row_id', 'pid', 'tcreated', 'tmodified', 'otype', 's', 'p', 'o', 'n', 'altids', 'geometry', 'authorized_by', 'has_feature_of_interest', 'affiliation', 'sampling_purpose', 'complies_with', 'project', 'alternate_identifiers', 'relationship', 'elevation', 'sample_identifier', 'dc_rights', 'result_time', 'contact_information', 'latitude', 'target', 'role', 'scheme_uri', 'is_part_of', 'scheme_name', 'name', 'longitude', 'obfuscated', 'curation_location', 'last_modified_time', 'access_constraints', 'place_name', 'description', 'label', 'thumbnail_url']

Sample row:
row_id                                                                     1
pid                          geoloc_1b9d137af503afe20873ccbd9797b64eb0d4bcfb
tcreated                                                                <NA>
tmodified                                                               <NA>
otype                                                GeospatialCoordLocation
s                           

In [12]:
def export_site_subgraph(conn, site_name_pattern, output_prefix):
    """Export all data related to a specific site"""
    
    # Find the site
    site_info = conn.execute("""
        SELECT row_id, pid, label
        FROM pqg
        WHERE otype = 'SamplingSite'
        AND label LIKE ?
        LIMIT 1
    """, [f'%{site_name_pattern}%']).fetchdf()
    
    if site_info.empty:
        print(f"No site found matching '{site_name_pattern}'")
        return None
    
    site_row_id = site_info.iloc[0]['row_id']
    print(f"Found site: {site_info.iloc[0]['label']}")
    
    # Get all related entities (simplified version - not recursive)
    related_data = conn.execute("""
        WITH site_related AS (
            -- Get the site itself
            SELECT * FROM pqg WHERE row_id = ?
            
            UNION ALL
            
            -- Get edges from the site
            SELECT * FROM pqg e
            WHERE e.otype = '_edge_' AND e.s = ?
            
            UNION ALL
            
            -- Get entities connected to the site
            SELECT n.* FROM pqg e
            JOIN pqg n ON n.row_id = e.o[1]
            WHERE e.otype = '_edge_' AND e.s = ?
        )
        SELECT * FROM site_related
    """, [site_row_id, site_row_id, site_row_id]).fetchdf()
    
    # Save to parquet
    output_file = f"{output_prefix}_{site_info.iloc[0]['pid']}.parquet"
    related_data.to_parquet(output_file)
    print(f"Exported {len(related_data)} rows to {output_file}")
    
    return related_data

# Example usage (commented out to avoid creating files)
# pompeii_data = export_site_subgraph(conn, "Pompeii", "pompeii_subgraph")

## Data Quality Analysis

In [13]:
# Check for location data quality
location_quality = conn.execute("""
    SELECT
        CASE 
            WHEN obfuscated THEN 'Obfuscated'
            ELSE 'Precise'
        END as location_type,
        COUNT(*) as count,
        AVG(CASE WHEN latitude IS NOT NULL THEN 1.0 ELSE 0.0 END) * 100 as pct_with_coords
    FROM pqg
    WHERE otype = 'GeospatialCoordLocation'
    GROUP BY location_type
""").fetchdf()

print("Location Data Quality:")
print(location_quality)

Location Data Quality:
  location_type   count  pct_with_coords
0       Precise  197221        99.999493
1    Obfuscated    1926       100.000000


In [14]:
# Check for orphaned nodes (nodes not connected by any edge)
orphan_check = conn.execute("""
    WITH connected_nodes AS (
        SELECT DISTINCT s as row_id FROM pqg WHERE otype = '_edge_'
        UNION
        SELECT DISTINCT unnest(o) as row_id FROM pqg WHERE otype = '_edge_'
    )
    SELECT
        n.otype,
        COUNT(*) as orphan_count
    FROM pqg n
    LEFT JOIN connected_nodes c ON n.row_id = c.row_id
    WHERE n.otype != '_edge_' AND c.row_id IS NULL
    GROUP BY n.otype
""").fetchdf()

print("\nOrphaned Nodes by Type:")
print(orphan_check if not orphan_check.empty else "No orphaned nodes found!")


Orphaned Nodes by Type:
               otype  orphan_count
0              Agent             1
1  IdentifiedConcept         16990


## Summary Statistics

In [15]:
# Generate comprehensive summary
summary = conn.execute("""
    WITH stats AS (
        SELECT
            COUNT(*) as total_rows,
            COUNT(DISTINCT pid) as unique_pids,
            COUNT(CASE WHEN otype = '_edge_' THEN 1 END) as edge_count,
            COUNT(CASE WHEN otype != '_edge_' THEN 1 END) as node_count,
            COUNT(DISTINCT CASE WHEN otype != '_edge_' THEN otype END) as entity_types,
            COUNT(DISTINCT p) as relationship_types
        FROM pqg
    )
    SELECT * FROM stats
""").fetchdf()

print("Dataset Summary:")
for col in summary.columns:
    print(f"{col}: {summary[col].iloc[0]:,}")

Dataset Summary:
total_rows: 11,786,509
unique_pids: 11,786,509
edge_count: 9,321,819
node_count: 2,464,690
entity_types: 6
relationship_types: 10


## Debug: Specific Geo Point Analysis

Testing queries for parquet_cesium.qmd debugging. This section demonstrates:
- **Generic PQG debugging**: How to trace edge connections
- **OpenContext validation**: Verifying archaeological data relationships

In [16]:
# Utility functions
def ark_to_url(pid: str) -> str:
    """Return a resolvable n2t.net URL for an ARK identifier.
    If pid is not an ARK, return it as a string.
    """
    if isinstance(pid, str) and pid.startswith("ark:/"):
        return f"https://n2t.net/{pid}"
    return str(pid)

def ensure_connection():
    """Ensure we have a working DuckDB connection with the parquet view"""
    global conn
    try:
        conn.execute("SELECT 1").fetchone()
    except (NameError, Exception):
        print("Recreating DuckDB connection...")
        conn = duckdb.connect()
        conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")
        print("Connection restored!")
    return conn

In [17]:
# Debug specific geo location from parquet_cesium.qmd
# This section remains provider-agnostic and uses iSamples model semantics

target_geo_pid = "geoloc_7ea562cce4c70e4b37f7915e8384880c86607729"

print(f"=== Debugging geo location: {target_geo_pid} ===\n")

# 1. Find the geo location record
geo_record = conn.execute("""
    SELECT row_id, pid, otype, latitude, longitude 
    FROM pqg 
    WHERE pid = ? AND otype = 'GeospatialCoordLocation'
""", [target_geo_pid]).fetchdf()

print("1. Geo Location Record:")
if not geo_record.empty:
    print(geo_record.to_dict('records')[0])
    geo_row_id = geo_record.iloc[0]['row_id']
    print(f"   Row ID: {geo_row_id}")
else:
    print("   ❌ Geo location not found!")
    geo_row_id = None

=== Debugging geo location: geoloc_7ea562cce4c70e4b37f7915e8384880c86607729 ===

1. Geo Location Record:
{'row_id': 194991, 'pid': 'geoloc_7ea562cce4c70e4b37f7915e8384880c86607729', 'otype': 'GeospatialCoordLocation', 'latitude': 28.058084, 'longitude': -81.146851}
   Row ID: 194991


In [18]:
# 2. Check what edges point to this geo location
if geo_row_id is not None:
    geo_row_id_int = int(geo_row_id)
    edges_to_geo = conn.execute("""
        SELECT s, p, otype as edge_type, pid as edge_pid
        FROM pqg 
        WHERE otype = '_edge_' AND ? = ANY(o)
    """, [geo_row_id_int]).fetchdf()

    print(f"\n2. Edges pointing to this geo location ({len(edges_to_geo)} found):")
    if not edges_to_geo.empty:
        edge_summary = edges_to_geo.groupby('p').size().reset_index()
        edge_summary.columns = ['predicate', 'count']
        print(edge_summary)
        print("\nDetailed edges:")
        for _, edge in edges_to_geo.iterrows():
            print(f"   {edge['p']}: row_id {edge['s']} -> geo location")
    else:
        print("   ❌ No edges point to this geo location!")
else:
    print("\n2. Skipping edge analysis - geo location not found")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


2. Edges pointing to this geo location (1 found):
       predicate  count
0  site_location      1

Detailed edges:
   site_location: row_id 217020 -> geo location


In [19]:
# 3. Direct event samples
if geo_row_id is not None:
    direct_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            s.name as sample_name,
            evt.pid as event_id,
            evt.label as event_label,
            'direct_event_location' as location_path
        FROM pqg s
        JOIN pqg e1  ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt ON e1.o[1] = evt.row_id
        JOIN pqg e2  ON evt.row_id = e2.s AND e2.p = 'sample_location'
        JOIN pqg g   ON e2.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 20
    """, [target_geo_pid]).fetchdf()

    print(f"\n3. Direct Event Samples ({len(direct_samples)} found):")
    if not direct_samples.empty:
        print(direct_samples[['sample_id', 'sample_label', 'event_id', 'event_label']].head())
    else:
        print("   ❌ No direct event samples found!")
else:
    print("\n3. Skipping direct samples query - geo location not found")


3. Direct Event Samples (0 found):
   ❌ No direct event samples found!


In [20]:
# 4. Site-associated samples
if geo_row_id is not None:
    site_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            s.name as sample_name,
            evt.pid as event_id,
            evt.label as event_label,
            site.label as site_name,
            'via_site_location' as location_path
        FROM pqg s
        JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt  ON e1.o[1] = evt.row_id
        JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sampling_site'
        JOIN pqg site ON e2.o[1] = site.row_id
        JOIN pqg e3   ON site.row_id = e3.s AND e3.p = 'site_location'
        JOIN pqg g    ON e3.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND site.otype = 'SamplingSite'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 20
    """, [target_geo_pid]).fetchdf()

    print(f"\n4. Site-Associated Samples ({len(site_samples)} found):")
    if not site_samples.empty:
        print(site_samples[['sample_id', 'sample_label', 'site_name', 'event_id']].head())
    else:
        print("   ❌ No site-associated samples found!")
else:
    print("\n4. Skipping site samples query - geo location not found")


4. Site-Associated Samples (1 found):
              sample_id    sample_label       site_name  \
0  ark:/28722/k2x63t42w  Assemblage 364  Osceola County   

                                            event_id  
0  sampevent_b19416f025a0b804563976f00aa78a8524c2...  


In [21]:
# 5. If we found samples, get detailed metadata for the first sample
all_samples = []
if 'direct_samples' in locals() and not direct_samples.empty:
    all_samples.extend(direct_samples.to_dict('records'))
if 'site_samples' in locals() and not site_samples.empty:
    all_samples.extend(site_samples.to_dict('records'))

if all_samples:
    first_sample = all_samples[0]
    sample_pid = first_sample['sample_id']

    print(f"\n5. Detailed metadata for sample: {sample_pid}")
    print(f"   Resolvable URL: {ark_to_url(sample_pid)}")
    print(f"   Sample label: {first_sample.get('sample_label', 'N/A')}")
    print(f"   Location path: {first_sample.get('location_path', 'N/A')}")

    # Materials for this sample
    materials = conn.execute("""
        SELECT DISTINCT
            mat.pid as material_id,
            mat.label as material_type,
            mat.name as material_category
        FROM pqg s
        JOIN pqg e   ON s.row_id = e.s AND e.p = 'has_material_category'
        JOIN pqg mat ON e.o[1] = mat.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND s.pid = ?
          AND e.otype = '_edge_'
          AND mat.otype = 'IdentifiedConcept'
    """, [sample_pid]).fetchdf()

    print(f"\n   Materials ({len(materials)} found):")
    if not materials.empty:
        for _, mat in materials.iterrows():
            print(f"     - {mat['material_type']} ({ark_to_url(mat['material_id'])})")
    else:
        print("     ❌ No materials found!")

    # Agents responsible for this sample
    agents = conn.execute("""
        SELECT DISTINCT
            agent.pid as agent_id,
            agent.label as agent_name,
            agent.name as agent_role
        FROM pqg s
        JOIN pqg e1    ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt   ON e1.o[1] = evt.row_id
        JOIN pqg e2    ON evt.row_id = e2.s AND e2.p = 'responsibility'
        JOIN pqg agent ON e2.o[1] = agent.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND s.pid = ?
          AND e1.otype = '_edge_'
          AND evt.otype = 'SamplingEvent'
          AND e2.otype = '_edge_'
          AND agent.otype = 'Agent'
        LIMIT 10
    """, [sample_pid]).fetchdf()

    print(f"\n   Responsible Agents ({len(agents)} found):")
    if not agents.empty:
        for _, agent in agents.iterrows():
            print(f"     - {agent['agent_name']} ({ark_to_url(agent['agent_id'])})")
    else:
        print("     ❌ No agents found!")
else:
    print("\n5. No samples found to analyze metadata")


5. Detailed metadata for sample: ark:/28722/k2x63t42w
   Resolvable URL: https://n2t.net/ark:/28722/k2x63t42w
   Sample label: Assemblage 364
   Location path: via_site_location

   Materials (1 found):
     - Material (https://w3id.org/isample/vocabulary/material/1.0/material)

   Responsible Agents (1 found):
     - None (https://opencontext.org/persons/ce3e13cb-c7b6-4d61-55fe-bb0d52a8374a)


In [22]:
# 6. Summary of findings for this geo location
print(f"\n=== SUMMARY for {target_geo_pid} ===")
if geo_row_id is not None:
    print(f"✅ Geo location found (row_id: {geo_row_id})")
    print(f"📍 Coordinates: {geo_record.iloc[0]['latitude']}, {geo_record.iloc[0]['longitude']}")

    total_samples = len(all_samples)
    direct_count = len([s for s in all_samples if s.get('location_path') == 'direct_event_location'])
    site_count = len([s for s in all_samples if s.get('location_path') == 'via_site_location'])

    print(f"🔬 Total samples found: {total_samples}")
    print(f"   - Direct event samples: {direct_count}")
    print(f"   - Site-associated samples: {site_count}")

    if total_samples > 0:
        print("✅ Sample metadata retrieval successful!")
    else:
        print("❌ No samples found for this location")
else:
    print("❌ Geo location not found in dataset!")

print(f"\n=== END DEBUG for {target_geo_pid} ===\n")


=== SUMMARY for geoloc_7ea562cce4c70e4b37f7915e8384880c86607729 ===
✅ Geo location found (row_id: 194991)
📍 Coordinates: 28.058084, -81.146851
🔬 Total samples found: 1
   - Direct event samples: 0
   - Site-associated samples: 1
✅ Sample metadata retrieval successful!

=== END DEBUG for geoloc_7ea562cce4c70e4b37f7915e8384880c86607729 ===



In [23]:
# 7. Test with a different geo location that has sample_location edges
sample_location_geos = conn.execute("""
    SELECT g.pid, g.latitude, g.longitude, COUNT(*) as edge_count
    FROM pqg e
    JOIN pqg g ON e.o[1] = g.row_id
    WHERE e.otype = '_edge_'
      AND e.p = 'sample_location'
      AND g.otype = 'GeospatialCoordLocation'
    GROUP BY g.pid, g.latitude, g.longitude
    ORDER BY edge_count DESC
    LIMIT 3
""").fetchdf()

print("=== Testing with geo locations that have direct sample_location edges ===")
print(sample_location_geos)

if not sample_location_geos.empty:
    test_geo_pid = sample_location_geos.iloc[0]['pid']
    print(f"\nTesting direct samples query with: {test_geo_pid}")

    test_direct_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            evt.pid as event_id,
            evt.label as event_label
        FROM pqg s
        JOIN pqg e1  ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt ON e1.o[1] = evt.row_id
        JOIN pqg e2  ON evt.row_id = e2.s AND e2.p = 'sample_location'
        JOIN pqg g   ON e2.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 5
    """, [test_geo_pid]).fetchdf()

    print(f"Direct samples found: {len(test_direct_samples)}")
    if not test_direct_samples.empty:
        print("✅ Direct event samples exist")
        print(test_direct_samples[['sample_id', 'sample_label', 'event_id']].head())
    else:
        print("❌ Still no direct event samples found")
else:
    print("❌ No geo locations with sample_location edges found")

=== Testing with geo locations that have direct sample_location edges ===
                                               pid   latitude  longitude  \
0  geoloc_35842a4fa478ae28c68f54d1db36c8e968d62dcb  37.668196  32.827191   
1  geoloc_17bae610b87227ef806161bdb40ac97b4cd8ef5e  30.328700  35.442100   
2  geoloc_045c25c9e19aeac434ef19616cf2130175cfd130  35.034889  32.421841   

   edge_count  
0      131022  
1      108846  
2       52252  

Testing direct samples query with: geoloc_35842a4fa478ae28c68f54d1db36c8e968d62dcb
Direct samples found: 5
✅ Direct event samples exist
              sample_id sample_label  \
0  ark:/28722/k2668d05z  16253.F1834   
1  ark:/28722/k2h991t2x     8864.F27   
2  ark:/28722/k2bv7f09q     1128.F82   
3  ark:/28722/k2cv4fm0m    15763.F26   
4  ark:/28722/k26m36047   16590.F831   

                                            event_id  
0  sampevent_44cdd124049ade33ac5da5b9a96ebe227a40...  
1  sampevent_84770032a3e6eef593a2f68f1da5f6cc98b5...  
2  sampevent_3

## Debug Analysis Results

### Key Findings for parquet_cesium.qmd

1. **Geo Location Structure**: The target geo location `geoloc_7ea562cce4c70e4b37f7915e8384880c86607729` exists in the dataset with correct coordinates.

2. **MaterialSampleRecord Association**: This specific location has **1 site-associated MaterialSampleRecord** but **0 direct event MaterialSampleRecord instances**.

3. **Query Validation**: Both query paths work correctly:
   - **Direct path**: `MaterialSampleRecord → SamplingEvent → sample_location → GeospatialCoordLocation`
   - **Site path**: `MaterialSampleRecord → SamplingEvent → SamplingSite → site_location → GeospatialCoordLocation`

4. **Data Availability**: The dataset contains both types of MaterialSampleRecord associations, but not every geo location has both types.

### Recommendations for parquet_cesium.qmd

- The JavaScript queries are correctly structured and should work
- Some geo locations may only have site-associated MaterialSampleRecord instances (like our test case)
- Consider showing both direct and site-associated MaterialSampleRecord instances in the UI
- Add debug logging to identify when no MaterialSampleRecord instances are found vs. query errors

In [24]:
# Analysis complete!
print("\nAnalysis complete!")
print("Note: DuckDB connection remains open for interactive use")


Analysis complete!
Note: DuckDB connection remains open for interactive use


## Read PQG key-value metadata (iSamples generic)

The parquet contains KV metadata describing the iSamples PQG schema (see https://github.com/isamplesorg/pqg). We’ll load the keys `pqg_version`, `pqg_primary_key`, `pqg_node_types`, `pqg_edge_fields`, `pqg_literal_fields` to make the notebook self‑describing and provider‑agnostic.

In [25]:
# Read PQG key-value metadata using PyArrow (provider-agnostic)
import pyarrow.parquet as pq

try:
    md = pq.read_metadata(parquet_path)
    kv_raw = md.metadata or {}
    # Decode byte keys/values to strings
    kv = { (k.decode() if isinstance(k, (bytes, bytearray)) else str(k)):
           (v.decode() if isinstance(v, (bytes, bytearray)) else str(v))
           for k, v in kv_raw.items() }

    wanted_keys = ["pqg_version", "pqg_primary_key", "pqg_node_types", "pqg_edge_fields", "pqg_literal_fields"]
    selected = {k: kv.get(k) for k in wanted_keys if k in kv}

    print("PQG KV metadata (selected):")
    if selected:
        for k in wanted_keys:
            if k in selected:
                print(f"- {k}: {selected[k][:120]}{'...' if len(selected[k])>120 else ''}")
    else:
        print("No PQG KV metadata keys found in file metadata")
except Exception as e:
    print("Unable to read parquet metadata via PyArrow:", e)

PQG KV metadata (selected):
- pqg_version: 0.2.0
- pqg_primary_key: pid
- pqg_node_types: {"Agent": {"name": "name VARCHAR DEFAULT NULL", "affiliation": "affiliation VARCHAR DEFAULT NULL", "contact_information"...
- pqg_edge_fields: ["pid", "otype", "s", "p", "o", "n", "altids", "geometry"]
- pqg_literal_fields: ["authorized_by", "has_feature_of_interest", "affiliation", "sampling_purpose", "complies_with", "project", "alternate_i...


In [26]:


# Count records
result = conn.execute("SELECT COUNT(*) FROM pqg;").fetchone()
result

(11786509,)

In [27]:
# Helper queries around a sample PID and a geo PID

# Path 1 (Direct event location):
#   MaterialSampleRecord -> produced_by -> SamplingEvent -> sample_location -> GeospatialCoordLocation

# Path 2 (Via site location):
#   MaterialSampleRecord -> produced_by -> SamplingEvent -> sampling_site -> SamplingSite -> site_location -> GeospatialCoordLocation

# Notes on the queries below:
# - The PQG table stores both nodes (MaterialSampleRecord, SamplingEvent, SamplingSite, GeospatialCoordLocation, etc.) and edges (otype = '_edge_').
# - WHERE and JOIN conditions enforce which path(s) are required for a row to appear.
# - Inner JOINs mean rows will only be returned when all joined paths/objects exist.


def get_sample_data_via_sample_pid(sample_pid, con, show_max_width):
    """Return one row of core sample metadata, including site and geo coordinates, for a sample PID.

    What it does
    - Starts at the MaterialSampleRecord identified by the given `sample_pid`.
    - Follows produced_by -> SamplingEvent.
    - Follows sample_location -> GeospatialCoordLocation to fetch latitude/longitude (Path 1).
    - Follows sampling_site -> SamplingSite to fetch site label and PID (Path 2).

    Important implications
    - This query uses INNER JOINs on BOTH the Path 1 and Path 2 chains. Therefore, it returns a row only if the sample has:
        1) a SamplingEvent with a sample_location pointing to a GeospatialCoordLocation (Path 1), and
        2) a SamplingEvent with a sampling_site pointing to a SamplingSite (Path 2).
      If either path is missing, the query returns no rows.

    Parameters
    - sample_pid (str): The iSamples PID of the MaterialSampleRecord to look up.
    - con: A DuckDB connection with the PQG table registered as `pqg`.
    - show_max_width: Width passed to DuckDB's .show() for display formatting.

    Returns
    - DuckDB relation (con.sql(sql)): The prepared relation; also prints a preview via .show().
    """

    sql = f"""
    SELECT 
        samp_pqg.row_id,
        samp_pqg.pid AS sample_pid,
        samp_pqg.alternate_identifiers AS sample_alternate_identifiers,
        samp_pqg.label AS sample_label,
        samp_pqg.description AS sample_description,
        samp_pqg.thumbnail_url AS sample_thumbnail_url,
        samp_pqg.thumbnail_url is NOT NULL as has_thumbnail,
        geo_pqg.latitude, 
        geo_pqg.longitude,
        site_pqg.label AS sample_site_label,
        site_pqg.pid AS sample_site_pid
    FROM pqg AS samp_pqg
    JOIN pqg AS samp_rel_se_pqg ON (samp_rel_se_pqg.s = samp_pqg.row_id AND samp_rel_se_pqg.p = 'produced_by')
    JOIN pqg AS se_pqg ON (list_extract(samp_rel_se_pqg.o, 1) = se_pqg.row_id AND se_pqg.otype = 'SamplingEvent')
    -- Path 1: event -> sample_location -> GeospatialCoordLocation
    JOIN pqg AS geo_rel_se_pqg ON (geo_rel_se_pqg.s = se_pqg.row_id AND geo_rel_se_pqg.p = 'sample_location')
    JOIN pqg AS geo_pqg ON (list_extract(geo_rel_se_pqg.o, 1) = geo_pqg.row_id AND geo_pqg.otype = 'GeospatialCoordLocation')
    -- Path 2: event -> sampling_site -> SamplingSite
    JOIN pqg AS site_rel_se_pqg ON (site_rel_se_pqg.s = se_pqg.row_id AND site_rel_se_pqg.p = 'sampling_site')
    JOIN pqg AS site_pqg ON (list_extract(site_rel_se_pqg.o, 1) = site_pqg.row_id AND site_pqg.otype = 'SamplingSite')
    WHERE samp_pqg.pid = '{sample_pid}' AND samp_pqg.otype = 'MaterialSampleRecord';
    """

    db_m = con.sql(sql)
    # db_m.show(max_width=show_max_width)
    return db_m


def get_sample_data_agents_sample_pid(sample_pid, con, show_max_width):
    """Return agent relationships (responsibility/registrant) for a sample PID.

    What it does
    - Starts at the MaterialSampleRecord identified by `sample_pid`.
    - Follows produced_by -> SamplingEvent.
    - From the event, follows predicates in ['responsibility', 'registrant'] to Agent nodes.

    Relationship to Path 1 vs Path 2
    - This query does NOT depend on Path 1 (direct geo) or Path 2 (via site). It only depends on the existence of the SamplingEvent and agent edges from that event. You will get agent rows even if the sample has no sample_location or sampling_site.

    Parameters
    - sample_pid (str): The sample PID.
    - con: DuckDB connection.
    - show_max_width: Width used by .show().

    Returns
    - DuckDB relation (con.sql(sql)): The prepared relation; also prints a preview via .show().
    """

    sql = f"""
    SELECT 
        samp_pqg.row_id,
        samp_pqg.pid AS sample_pid,
        samp_pqg.alternate_identifiers AS sample_alternate_identifiers,
        samp_pqg.label AS sample_label,
        samp_pqg.description AS sample_description,
        samp_pqg.thumbnail_url AS sample_thumbnail_url,
        samp_pqg.thumbnail_url is NOT NULL as has_thumbnail,
        agent_rel_se_pqg.p AS predicate,
        agent_pqg.pid AS agent_pid,
        agent_pqg.name AS agent_name,
        agent_pqg.alternate_identifiers AS agent_alternate_identifiers
    FROM pqg AS samp_pqg
    JOIN pqg AS samp_rel_se_pqg ON (samp_rel_se_pqg.s = samp_pqg.row_id AND samp_rel_se_pqg.p = 'produced_by')
    JOIN pqg AS se_pqg ON (list_extract(samp_rel_se_pqg.o, 1) = se_pqg.row_id AND se_pqg.otype = 'SamplingEvent')
    JOIN pqg AS agent_rel_se_pqg ON (agent_rel_se_pqg.s = se_pqg.row_id AND list_contains(['responsibility', 'registrant'], agent_rel_se_pqg.p))
    JOIN pqg AS agent_pqg ON (agent_pqg.row_id = ANY(agent_rel_se_pqg.o) AND agent_pqg.otype = 'Agent')
    WHERE samp_pqg.pid = '{sample_pid}' AND samp_pqg.otype = 'MaterialSampleRecord';
    """

    db_m = con.sql(sql)
    # db_m.show(max_width=show_max_width)
    return db_m


def get_sample_types_and_keywords_via_sample_pid(sample_pid, con, show_max_width):
    """Return IdentifiedConcept terms (keywords, object types, material categories) for a sample PID.

    What it does
    - Starts at the MaterialSampleRecord identified by `sample_pid`.
    - Follows predicates in ['keywords', 'has_sample_object_type', 'has_material_category'] to IdentifiedConcept nodes and returns their PID/label.

    Relationship to Path 1 vs Path 2
    - This query attaches concepts directly to the MaterialSampleRecord. It does not require Path 1 or Path 2 to exist and will return rows even if no geo/site relationships are present for the sample.

    Parameters
    - sample_pid (str): The sample PID.
    - con: DuckDB connection.
    - show_max_width: Width used by .show().

    Returns
    - DuckDB relation (con.sql(sql)): The prepared relation; also prints a preview via .show().
    """

    sql = f"""
    SELECT 
        samp_pqg.row_id,
        samp_pqg.pid AS sample_pid,
        samp_pqg.alternate_identifiers AS sample_alternate_identifiers,
        samp_pqg.label AS sample_label,
        kw_rel_se_pqg.p AS predicate,
        kw_pqg.pid AS keyword_pid,
        kw_pqg.label AS keyword
    FROM pqg AS samp_pqg
    JOIN pqg AS kw_rel_se_pqg ON (kw_rel_se_pqg.s = samp_pqg.row_id AND list_contains(['keywords', 'has_sample_object_type', 'has_material_category'], kw_rel_se_pqg.p))
    JOIN pqg AS kw_pqg ON (kw_pqg.row_id = ANY(kw_rel_se_pqg.o) AND kw_pqg.otype = 'IdentifiedConcept')
    WHERE samp_pqg.pid = '{sample_pid}' AND samp_pqg.otype = 'MaterialSampleRecord';
    """

    db_m = con.sql(sql)
    # db_m.show(max_width=show_max_width)
    return db_m


def get_samples_at_geo_cord_location_via_sample_event(geo_loc_pid, con, show_max_width):
    """Return samples anchored at a GeospatialCoordLocation PID via event sample_location, plus site info.

    What it does
    - Starts at a GeospatialCoordLocation identified by `geo_loc_pid`.
    - Follows incoming edges with p = 'sample_location' to reach SamplingEvent rows (Path 1 from the perspective of event -> geo; here we walk it in reverse starting at geo).
    - From each event, follows produced_by (reverse) to find MaterialSampleRecord rows produced by it.
    - Also enriches each event with its sampling_site -> SamplingSite to return site label/PID (Path 2).

    Relationship to Path 1 vs Path 2
    - Path 1 is REQUIRED because we start from the GeospatialCoordLocation and look for events that point to it via sample_location. Those events are then used to find samples produced by them.
    - Path 2 is JOINED to provide site context. Because the SQL uses INNER JOINs for site, only events that also have a SamplingSite will surface here. If you want direct-only results regardless of whether an event has a SamplingSite, change the site joins to LEFT JOINs.

    Parameters
    - geo_loc_pid (str): The PID of the GeospatialCoordLocation.
    - con: DuckDB connection.
    - show_max_width: Width used by .show().

    Returns
    - DuckDB relation (con.sql(sql)): The prepared relation; also prints a preview via .show().
    """

    sql = f"""
    SELECT geo_pqg.latitude, geo_pqg.longitude, 
           site_pqg.label AS sample_site_label,
           site_pqg.pid AS sample_site_pid,
           samp_pqg.pid AS sample_pid,
           samp_pqg.alternate_identifiers AS sample_alternate_identifiers,
           samp_pqg.label AS sample_label,
           samp_pqg.description AS sample_description,
           samp_pqg.thumbnail_url AS sample_thumbnail_url,
           samp_pqg.thumbnail_url is NOT NULL as has_thumbnail 
    FROM pqg AS geo_pqg
    JOIN pqg AS rel_se_pqg ON (rel_se_pqg.p = 'sample_location' AND contains(rel_se_pqg.o, geo_pqg.row_id))
    JOIN pqg AS se_pqg ON (rel_se_pqg.s = se_pqg.row_id AND se_pqg.otype = 'SamplingEvent')
    -- Path 2 enrichment: event -> sampling_site -> SamplingSite
    JOIN pqg AS rel_site_pqg ON (se_pqg.row_id = rel_site_pqg.s AND rel_site_pqg.p = 'sampling_site')
    JOIN pqg AS site_pqg ON (list_extract(rel_site_pqg.o, 1) = site_pqg.row_id AND site_pqg.otype = 'SamplingSite')
    -- Find samples produced by the event
    JOIN pqg AS rel_samp_pqg ON (rel_samp_pqg.p = 'produced_by' AND contains(rel_samp_pqg.o, se_pqg.row_id))
    JOIN pqg AS samp_pqg ON (rel_samp_pqg.s = samp_pqg.row_id AND samp_pqg.otype = 'MaterialSampleRecord')
    WHERE geo_pqg.pid = '{geo_loc_pid}' AND geo_pqg.otype = 'GeospatialCoordLocation'
    ORDER BY has_thumbnail DESC
    """

    db_m = con.sql(sql)
    # db_m.show(max_width=show_max_width)
    return db_m



In [28]:
sample_pid = "geoloc_7ea562cce4c70e4b37f7915e8384880c86607729"
sample_pid = "ark:/28722/k2xd0t39r"
get_sample_data_via_sample_pid(sample_pid, conn, 120)

┌─────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬───────────────┬───────────┬───────────┬───────────────────┬───────────────────────────────────────────────────────────────────────┐
│ row_id  │      sample_pid      │                                 sample_alternate_identifiers                                  │ sample_label │                                                   sample_description                                                   │ sample_thumbnail_url │ has_thumbnail │ latitude  │ longitude │ sample_site_label │                            sample_site_pid                            │
│  int32  │       varchar        │                                           varchar[]                                           │   varchar    │           

In [29]:
def get_sampling_sites_by_name(conn, site_name_pattern: str, use_wildcard: bool = True):
    """
    Retrieve all SamplingSites matching a given name pattern.
    
    Parameters
    - conn: DuckDB connection
    - site_name_pattern: Name or partial name to search for
    - use_wildcard: If True, wraps pattern with % for SQL LIKE; if False, matches exactly
    
    Returns DataFrame with columns:
    - site_row_id, site_pid, site_label, sample_count, unique_geo_count, event_count
    """
    ensure_connection()
    
    if use_wildcard:
        pattern = f"%{site_name_pattern}%"
    else:
        pattern = site_name_pattern
    
    sql = """
        SELECT
            site.row_id AS site_row_id,
            site.pid   AS site_pid,
            site.label AS site_label,
            COUNT(DISTINCT samp.row_id) AS sample_count,
            COUNT(DISTINCT geo.pid) AS unique_geo_count,
            COUNT(DISTINCT evt.row_id) AS event_count
        FROM pqg site
        LEFT JOIN pqg e_ss   ON (e_ss.p = 'sampling_site' AND e_ss.o[1] = site.row_id)
        LEFT JOIN pqg evt    ON (evt.row_id = e_ss.s AND evt.otype = 'SamplingEvent')
        LEFT JOIN pqg e_pb   ON (e_pb.p = 'produced_by' AND e_pb.o[1] = evt.row_id)
        LEFT JOIN pqg samp   ON (samp.row_id = e_pb.s AND samp.otype = 'MaterialSampleRecord')
        LEFT JOIN pqg e_geo  ON (e_geo.p = 'site_location' AND e_geo.s = site.row_id)
        LEFT JOIN pqg geo    ON (geo.row_id = e_geo.o[1] AND geo.otype = 'GeospatialCoordLocation')
        WHERE site.otype = 'SamplingSite'
          AND site.label LIKE ?
        GROUP BY site.row_id, site.pid, site.label
        ORDER BY sample_count DESC
    """
    
    return conn.execute(sql, [pattern]).fetchdf()

In [30]:
get_sample_data_agents_sample_pid(sample_pid, conn, 120)

┌─────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬───────────────┬────────────────┬──────────────────────────────────────────────────────────────────────┬────────────────┬─────────────────────────────┐
│ row_id  │      sample_pid      │                                 sample_alternate_identifiers                                  │ sample_label │                                                   sample_description                                                   │ sample_thumbnail_url │ has_thumbnail │   predicate    │                              agent_pid                               │   agent_name   │ agent_alternate_identifiers │
│  int32  │       varchar        │                                           varchar[]                                

In [31]:
get_sample_types_and_keywords_via_sample_pid(sample_pid, conn, 120)

┌─────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┬────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────────────────────────┐
│ row_id  │      sample_pid      │                                 sample_alternate_identifiers                                  │ sample_label │       predicate        │                                  keyword_pid                                  │            keyword            │
│  int32  │       varchar        │                                           varchar[]                                           │   varchar    │        varchar         │                                    varchar                                    │            varchar            │
├─────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────┼──────

In [32]:
get_samples_at_geo_cord_location_via_sample_event(sample_pid, conn, 120)

┌──────────┬───────────┬───────────────────┬─────────────────┬────────────┬──────────────────────────────┬──────────────┬────────────────────┬──────────────────────┬───────────────┐
│ latitude │ longitude │ sample_site_label │ sample_site_pid │ sample_pid │ sample_alternate_identifiers │ sample_label │ sample_description │ sample_thumbnail_url │ has_thumbnail │
│  double  │  double   │      varchar      │     varchar     │  varchar   │          varchar[]           │   varchar    │      varchar       │       varchar        │    boolean    │
├──────────┴───────────┴───────────────────┴─────────────────┴────────────┴──────────────────────────────┴──────────────┴────────────────────┴──────────────────────┴───────────────┤
│                                                                                      0 rows                                                                                       │
└─────────────────────────────────────────────────────────────────────────────────────────

In [33]:
%load_ext sql

Tip: You may define configurations in /Users/raymondyee/C/src/iSamples/isamples-python/pyproject.toml or /Users/raymondyee/.jupysql/config.

Did not find user configurations in /Users/raymondyee/C/src/iSamples/isamples-python/pyproject.toml.

In [34]:
# Connect to an in-memory DuckDB instance using %sql magic
%sql duckdb:///:memory:

# Create a view for the Parquet file (run this only once per session)
%sql CREATE VIEW pqg AS SELECT * FROM '/Users/raymondyee/Data/iSample/pqg_refining/oc_isamples_pqg.parquet';

Connecting to 'duckdb:///:memory:'

Running query in 'duckdb:///:memory:'

RuntimeError: (duckdb.duckdb.IOException) IO Error: No files found that match the pattern "/Users/raymondyee/Data/iSample/pqg_refining/oc_isamples_pqg.parquet"
[SQL: CREATE VIEW pqg AS SELECT * FROM '/Users/raymondyee/Data/iSample/pqg_refining/oc_isamples_pqg.parquet' ;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
%%sql

# count the number of rows in pqg
SELECT COUNT(*) FROM pqg;

In [ ]:
# Query geolocation records (pid, latitude, longitude) associated with PKAP Survey Area
ensure_connection()

pkap_geos = conn.execute("""
    SELECT
        site.pid   AS site_pid,
        site.label AS site_label,
        geo.pid    AS geo_pid,
        geo.row_id AS geo_row_id,
        geo.latitude,
        geo.longitude
    FROM pqg site
    JOIN pqg rel  ON (rel.s = site.row_id AND rel.p = 'site_location')
    JOIN pqg geo  ON (rel.o[1] = geo.row_id AND geo.otype = 'GeospatialCoordLocation')
    WHERE site.otype = 'SamplingSite'
      AND site.label = 'PKAP Survey Area'
    ORDER BY geo.pid
""").fetchdf()

print(f"Found {len(pkap_geos):,} geolocations for PKAP Survey Area")
pkap_geos.head(50)

In [ ]:
pkap_geoloc_id = "geoloc_ff64156b561ebb054e43183135f46f8c30f7e526"
get_samples_at_geo_cord_location_via_sample_event(pkap_geoloc_id, conn, 120)

In [ ]:
%%sql

-- Check whether all sampling sites have exactly one associated geolocation
WITH site_geo_counts AS (
    SELECT
        site.pid AS site_id,
        COUNT(DISTINCT geo.pid) AS geo_count
    FROM pqg site
    JOIN pqg e ON site.row_id = e.s AND e.p = 'site_location'
    JOIN pqg geo ON e.o[1] = geo.row_id AND geo.otype = 'GeospatialCoordLocation'
    WHERE site.otype = 'SamplingSite'
    GROUP BY site.pid
)
SELECT
    CASE WHEN MIN(geo_count) = 1 AND MAX(geo_count) = 1 THEN 'Yes' ELSE 'No' END AS all_sites_exactly_one_geo
FROM site_geo_counts;


### Query: GeospatialCoordLocation linked to both SamplingEvent and SamplingSite

This query finds geographic points (GeospatialCoordLocation) that have incoming edges from both:
- SamplingEvent via `sample_location` (Path 1)
- SamplingSite via `site_location` (Path 2)

It returns the geo PID, coordinates, and the counts of each edge type.

In [ ]:
# Find GeospatialCoordLocation nodes connected to both SamplingEvent (sample_location) and SamplingSite (site_location)
ensure_connection()

both_paths_geos = conn.execute("""
    WITH event_geos AS (
        SELECT g.row_id AS geo_row_id, g.pid AS geo_pid
        FROM pqg e
        JOIN pqg g ON e.o[1] = g.row_id
        WHERE e.otype = '_edge_'
          AND e.p = 'sample_location'
          AND g.otype = 'GeospatialCoordLocation'
    ),
    site_geos AS (
        SELECT g.row_id AS geo_row_id, g.pid AS geo_pid
        FROM pqg e
        JOIN pqg g ON e.o[1] = g.row_id
        WHERE e.otype = '_edge_'
          AND e.p = 'site_location'
          AND g.otype = 'GeospatialCoordLocation'
    ),
    event_counts AS (
        SELECT g.row_id AS geo_row_id, COUNT(*) AS sample_location_edges
        FROM pqg g
        JOIN pqg e ON e.o[1] = g.row_id AND e.otype = '_edge_' AND e.p = 'sample_location'
        WHERE g.otype = 'GeospatialCoordLocation'
        GROUP BY g.row_id
    ),
    site_counts AS (
        SELECT g.row_id AS geo_row_id, COUNT(*) AS site_location_edges
        FROM pqg g
        JOIN pqg e ON e.o[1] = g.row_id AND e.otype = '_edge_' AND e.p = 'site_location'
        WHERE g.otype = 'GeospatialCoordLocation'
        GROUP BY g.row_id
    )
    SELECT g.pid, g.latitude, g.longitude,
           COALESCE(ec.sample_location_edges, 0) AS sample_location_edges,
           COALESCE(sc.site_location_edges, 0) AS site_location_edges
    FROM pqg g
    JOIN event_geos eg ON eg.geo_row_id = g.row_id
    JOIN site_geos sg ON sg.geo_row_id = g.row_id
    LEFT JOIN event_counts ec ON ec.geo_row_id = g.row_id
    LEFT JOIN site_counts sc ON sc.geo_row_id = g.row_id
    WHERE g.otype = 'GeospatialCoordLocation'
    ORDER BY (COALESCE(ec.sample_location_edges, 0) + COALESCE(sc.site_location_edges, 0)) DESC
    LIMIT 50
""").fetchdf()

print(f"GeospatialCoordLocation linked to both paths: {len(both_paths_geos)} found")
both_paths_geos.head(10)

In [ ]:
# PKAP Survey Area

# calculate site_id for "PKAP Survey Area"
k = get_sampling_sites_by_name(conn, "PKAP Survey Area", use_wildcard=False)
if len(k) == 1:
    site_id = k.iloc[0]['site_pid']
    site_row_id = int(k.iloc[0]['site_row_id'])
    print(f"✅ Found exact match for 'PKAP Survey Area': {site_id}")


site_geos = conn.execute("""
    SELECT geo.pid, geo.latitude, geo.longitude
    FROM pqg e
    JOIN pqg geo ON e.o[1] = geo.row_id AND geo.otype = 'GeospatialCoordLocation'
    WHERE e.otype = '_edge_' AND e.p = 'site_location' AND e.s = ?
""", [site_row_id]).fetchdf()
print(f"Found {len(site_geos):,} geolocations for site '{site_id}'")
site_geos.head(10)
# pkap_geoloc_id = "geoloc_ff64156b561ebb054e43183135f46f8c30f7e526"

# I'm expecting site_geos to be only one row, matching pkap_geoloc_id
geo_pid = site_geos.iloc[0]['pid']
if geo_pid == pkap_geoloc_id:
    print(f"✅ Geolocation PID matches expected: {geo_pid}")

In [ ]:
geo_pid

# what functions take geo_pid as input?
get_samples_at_geo_cord_location_via_sample_event(geo_pid, conn, 120)


In [ ]:
# Test the updated get_sampling_sites_by_name function with row_id
k = get_sampling_sites_by_name(conn, "PKAP Survey Area", use_wildcard=False)
print("Columns returned:", list(k.columns))
print("\nSample row:")
k.head(1)

In [ ]:
# Example: Using site_row_id for efficient joins
# Previously, you'd need to use site_pid for joins. Now site_row_id is available!

site_info = k[['site_row_id', 'site_pid', 'site_label', 'sample_count']].iloc[0]
print(f"Site: {site_info['site_label']}")
print(f"  Row ID: {site_info['site_row_id']} (now directly usable for joins)")
print(f"  PID: {site_info['site_pid']}")
print(f"  Samples: {site_info['sample_count']:,}")

# You can now use site_row_id directly in follow-up queries without needing to join via PID
site_row_id_value = site_info['site_row_id']
print(f"\n✅ site_row_id = {site_row_id_value} is ready for direct use in SQL joins!")